In [59]:
import re
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import pickle

In [60]:
nltk.download('stopwords')

#  Load Dataset (Make sure the path is correct)
dataset = pd.read_csv(r"Restaurant_Reviews.tsv", delimiter='\t', quoting=3)



[nltk_data] Downloading package stopwords to C:\Users\Ritvik
[nltk_data]     Bhardwaj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [61]:
def clean_text(text):
    stopwords_set = set(stopwords.words('english')) - {'not'}
    text = re.sub(r'[^a-zA-Z]', ' ', text)  
    text = text.lower().split()
    text = [word for word in text if word not in stopwords_set]  # Stopwords remove (except "not")
    return ' '.join(text)


In [62]:
corpus = [clean_text(review) for review in dataset['Review'].values]

In [63]:
MAX_VOCAB = 10000  # Max unique words
MAX_LENGTH = 150  # Max review length

In [64]:
tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<OOV>")
tokenizer.fit_on_texts(corpus)
X = tokenizer.texts_to_sequences(corpus)
X = pad_sequences(X, maxlen=MAX_LENGTH, padding='post')

In [65]:
y = dataset['Liked'].values
print(y)

[1 0 0 1 1 0 0 0 1 1 1 0 0 1 0 0 1 0 0 0 0 1 1 1 1 1 0 1 0 0 1 0 1 0 1 1 1
 0 1 0 1 0 0 1 0 1 0 1 1 1 1 1 1 0 1 1 0 0 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 0
 0 0 0 1 1 0 0 0 0 1 0 1 0 1 1 1 0 1 0 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0
 0 1 1 1 1 0 0 0 0 0 0 1 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 0 1 0 0 0 0 1 1 0 0
 0 0 1 1 0 0 1 1 1 1 1 0 0 1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 0 0 0 0 0 1
 1 0 1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0
 0 0 0 1 1 1 0 1 1 0 1 0 1 0 0 1 0 1 0 1 0 0 0 0 1 1 1 0 1 0 1 0 1 1 1 0 1
 0 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 0 0 1 0 0 0 1 1 0 0 1 0 0 0 1 0 1 1
 0 1 0 1 1 0 0 0 1 0 0 0 1 1 1 0 1 0 1 0 0 1 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1
 0 1 1 0 0 1 0 0 1 1 1 0 1 1 1 1 1 0 0 1 0 1 1 0 1 1 1 0 1 1 0 1 0 0 1 1 1
 0 0 1 1 0 1 0 1 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 0 1
 1 1 0 1 1 0 1 0 0 0 1 1 1 1 0 0 0 0 1 1 0 0 1 0 1 1 0 1 0 1 1 1 1 0 1 1 0
 1 1 0 0 1 1 0 1 0 0 0 0 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 0 1 0 0 0 1 1 1 1 0
 1 0 0 1 1 1 0 0 1 1 1 0 

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [67]:
from tensorflow.keras.layers import Bidirectional

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=128, input_length=MAX_LENGTH),
    Bidirectional(LSTM(100, return_sequences=True)),  # Bidirectional LSTM
    Dropout(0.2),
    Bidirectional(LSTM(50)),  # Another LSTM layer
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])


c:\Users\Ritvik Bhardwaj\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [68]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [69]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 172ms/step - accuracy: 0.5484 - loss: 0.6944 - val_accuracy: 0.5250 - val_loss: 0.6849
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 126ms/step - accuracy: 0.6735 - loss: 0.6323 - val_accuracy: 0.7650 - val_loss: 0.6042
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 116ms/step - accuracy: 0.9374 - loss: 0.2196 - val_accuracy: 0.7400 - val_loss: 0.6984
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - accuracy: 0.8937 - loss: 0.2665 - val_accuracy: 0.7500 - val_loss: 0.6834
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 0.9697 - loss: 0.1154 - val_accuracy: 0.7550 - val_loss: 0.8796
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - accuracy: 0.9461 - loss: 0.1584 - val_accuracy: 0.7450 - val_loss: 0.6658
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - accuracy: 0.9684 - loss: 0.1147 - val_accuracy: 0.7600 - val_loss: 0.7059
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 0.9686 - loss: 0.0870 - val_accuracy: 0

In [70]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.7471 - loss: 0.8845
Test Accuracy: 0.7500


In [71]:
model.save("lstm_sentiment_model.h5")
with open("tokenizer.pkl", "wb") as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)

print("LSTM Model & Tokenizer saved successfully!")

LSTM Model & Tokenizer saved successfully!
